<a href="https://colab.research.google.com/github/mgnarag/machine-learning/blob/main/Testing_and_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/CS284/Project/"

In [6]:
import time
import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


from PIL import Image, ImageOps
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

import cv2

In [20]:
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def calculate_psnr(original, compressed):
    # Convert images to NumPy arrays
    original_np = np.array(original)
    compressed_np = np.array(compressed)

    # Calculate PSNR
    psnr = peak_signal_noise_ratio(original_np, compressed_np)
    return psnr

def calculate_ssim(original, compressed):
    # Convert images to NumPy arrays
    original_np = np.array(original)
    compressed_np = np.array(compressed)

    # Ensure images have the same shape
    if original_np.shape != compressed_np.shape:
        raise ValueError("Images must have the same dimensions for SSIM calculation")

    # Calculate SSIM
    ssim_value, _ = structural_similarity(original_np, compressed_np, full=True)
    return ssim_value

In [21]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np

def crop(im,n_size):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


filename = str("[GT] dirty.png")
test = Image.open(file_path+ "Testing nbs/" + filename)
test = test.convert('RGB')
w_dirty, h_dirty = test.size
gt = Image.open(file_path+ "Testing nbs/[GT] clean.png")
gt = gt.convert('RGB')

architecture = ['unet_','autoencoder_']
size = ['32_', '64_']
section = ['whole','portion']

models = []
for m in range(0,2):
  for s in range(0,2):
    for c in range(0,2):
      models.append(str(architecture[m]+size[s]+section[c]))

METRIC = []
for mod in range(0,8):
  if mod in [0, 1, 4, 5]:
    n_size = 32
    print(n_size)
  if mod in [2, 3, 6, 7]:
    n_size = 64
    print(n_size)

  model = keras.models.load_model(file_path + models[mod])
  print(model)
  xx = int(w_dirty/n_size)
  final=[]

  for portion in range(0,xx):
      im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
      w1, h1 = im1.size
      w = int(w1/n_size)
      h = int(h1/n_size)

      neverbeforeseen = np.array(crop(im1,n_size))
      #encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
      decoded_imgs = model.predict(neverbeforeseen)

      col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
      for i in range(2,h):
          col = np.vstack((col,decoded_imgs[i]))

      #y = np.where(col > 0.5,1,0) #round the values
      y= col
      y = (y * 255).astype('uint8')
      if portion == 0:
          final = y
      if portion > 0:
          final = np.hstack((final,y))

  final = np.squeeze(final)
  reconstructed = Image.fromarray(final)
  psnr = calculate_psnr(reconstructed, gt)
  psnr = calculate_ssim(reconstructed, gt)



  reconstructed.save(file_path + "Testing nbs/"+str(models[mod])+"_cleaned.png")

METRIC.append([models[mod],psnr,ssim])
METRIC = pd.DataFrame(METRIC)
headers = ['Model','PSNR','SSIM' ]
METRIC.columns = headers
METRIC.to_csv(file_path + 'Metrics.csv')
print(METRIC)

32
4/4 [==============================] - 0s 6ms/step


ValueError: Input images must have the same dimensions.